<a href="https://colab.research.google.com/github/WinetraubLab/coregister-xy/blob/fix-file-names/coregister_xy_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://github.com/WinetraubLab/coregister-xy/blob/main/coregister_xy_2.ipynb" target="_blank">
  <img src="https://img.shields.io/badge/view%20in-GitHub-blue" alt="View in GitHub"/>
</a>

# Overview
Use this notebook to get alignment information from ImageJ image registration. Print stats for individual barcodes and calculate mapping from angled tissue slice to flat view.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# @title Notebook Inputs { display-mode: "form" }
## @markdown How to use this notebook: [See Instructions]()
import numpy as np

# @markdown Input Paths:
# @markdown Leave either path blank to load a file from local file system.
trakem_xml_path = "/content/drive/Shareddrives/Yolab - Current Projects/_Datasets/2024-09-04 Multiple Barcode Alignment/align8.xml" # @param {type:"string"}
fluorescent_patch_number = 8 # @param {type:"integer"}
# @markdown For the alignment of multiple templates to one fluorescent image, specify the patch numbers of each template in the TrakEM stack.
# template_patch_1 = 11 # @param {type:"integer"}
# template_patch_2 = 14 # @param {type:"integer"}
# template_patch_3 = 17 # @param {type:"integer"}

# @markdown Enter template patch IDs in order, separated by commas. Example: 11, 14, 17
template_patch_list = "11, 14, 17, 20, 23, 26, 29, 32" # @param {type:"string"}

# @markdown Z-depth of each template, in um. Enter as a comma-separated list. Example: 50, 52, 54
template_z_list = "32, 34, 32, 32, 34, 34, 34, 36" # @param {type:"string"}

# @markdown Real (x,y) locations of photobleach barcode centers, as specified by script used to photobleach. Enter in format: (1000,0), (0, 1000), (0,0target_centersters = [0,1000], [1000, 1000], [0, 0] # @partarget_centersenters = np.target_centers_ctarget_centerset_centers = np.colutarget_centersrget_centertarget_centerstarget_centers.shape[0])))
target_centers = [0,0], [1000, 0], [2000, 0], [0,1000], [1000, 1000], [2000,1000],  [1000, 2000], [2000,2000] # @param

target_centers = np.array(target_centers)
assert target_centers.shape[1] == 2, "Points in target_centers should be in format [x,y]"

template_size = 401
um_per_pixel = 2

from google.colab import drive
from google.colab import files
# drive.mount('/content/drive/')

assert len(template_patch_list) == len(template_z_list), "Number of elements in template patch list and template z list must match"

if not trakem_xml_path:
  print("Upload saved TrakEM project:")
  uploaded = files.upload()
  trakem_xml_path = list(uploaded.keys())[0]
  trakem_xml_path = os.path.join(os.getcwd(), trakem_xml_path)


In [3]:
# @title Environment Setup
!git clone -b fix-file-names https://github.com/WinetraubLab/coregister-xy.git
%cd coregister-xy

from plane.fit_template import FitTemplate
from plane.fit_plane import FitPlane
import matplotlib.pyplot as plt
import os
from google.colab import files
import math
import pandas as pd
import numpy as np

%cd ..

Cloning into 'coregister-xy'...
remote: Enumerating objects: 664, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 664 (delta 101), reused 83 (delta 69), pack-reused 530 (from 1)
Receiving objects: 100% (664/664), 1.08 MiB | 13.78 MiB/s, done.
Resolving deltas: 100% (408/408), done.
/content/coregister-xy
/content


In [4]:
fps = []
templates = [int(x) for x in template_patch_list.split(",")]

for i in range(0, len(templates)):
    fp = FitTemplate.from_imagej_xml(trakem_xml_path, fluorescent_patch_number, templates[i], None, True)
    fps.append(fp)

uv_px = []
for i, fp in enumerate(fps):
    uv_px.append((fp.tx + template_size/2, fp.ty + template_size/2))

zs = np.array([float(x) for x in template_z_list.split(",")])
template_centers_xyz = np.column_stack((target_centers, zs))
template_centers_uv = np.array(uv_px)

multi_plane = FitPlane.from_template_centers(template_centers_uv, template_centers_xyz, print_inputs = False)

In [12]:
def get_single_template_stats(fps):
    """
    Prints stats for each FitPlane as a table: shrinkage, rotation, shear, and mean/stdev for each
    Units: um
    """
    num_templates = len(fps)
    projects_data = {
    "Template ID": [i for i in range(1, num_templates+1)],
    "Center (x)": [project.tx + template_size/2 for project in fps],
    "Center (y)": [project.ty + template_size/2 for project in fps],
    "Rotation (deg)": [project.theta_deg for project in fps],
    "Scaling": [project.scale for project in fps],
    "Shear magnitude": [project.shear_magnitude for project in fps],
    "Shear unit vector (x)": [project.shear_vector[0] for project in fps],
    "Shear unit vector (y)": [project.shear_vector[1] for project in fps]
    }

    columns_to_summarize = ["Rotation (deg)", "Scaling", "Shear magnitude", "Shear unit vector (x)", "Shear unit vector (y)"]

    # Create DataFrame
    df = pd.DataFrame(projects_data)

    # Compute mean and standard deviation for selected columns only
    mean_row = df[columns_to_summarize].mean()
    std_row = df[columns_to_summarize].std()

    # Append mean and std as new rows for selected columns only
    summary_df = df.copy()
    summary_df.loc['Mean', columns_to_summarize] = mean_row
    summary_df.loc['StDev', columns_to_summarize] = std_row
    summary_df = summary_df.round(2)
    summary_df = summary_df.replace(np.nan, '', regex=True)

    return summary_df


In [13]:
df = get_single_template_stats(fps)
df

,Template ID,Center (x),Center (y),Rotation (deg),Scaling,Shear magnitude,Shear unit vector (x),Shear unit vector (y)
0,1.0,644.31,1062.96,11.78,1.66,0.04,0.80,0.60
1,2.0,1454.47,1166.28,10.43,1.77,0.03,0.73,0.68
2,3.0,2425.81,1332.18,11.99,1.83,0.14,0.99,0.11
3,4.0,532.06,1851.14,13.16,1.79,0.07,1.00,0.02
4,5.0,1360.98,2003.33,9.54,1.88,0.07,0.96,0.28
5,6.0,2300.31,2129.16,11.40,1.93,0.04,0.95,0.32
6,7.0,1190.89,2833.23,13.24,1.86,0.09,1.00,0.08
7,8.0,2125.52,3012.87,11.73,1.97,0.06,0.95,-0.30
Mean,,,,11.66,1.84,0.07,0.92,0.22
StDev,,,,1.25,0.10,0.04,0.10,0.32


In [15]:
print("Template centers, in px:")
print(template_centers_uv)
print()

print("Template centers, in um:")
print(template_centers_xyz)
print()

print("Transformation coefficients:")
print("Vector u:", multi_plane.u)
print("Vector v:", multi_plane.v)
print("Vector h:", multi_plane.h)

Template centers, in px:
[[ 644.31455444 1062.9576679 ]
 [1454.47491053 1166.27868962]
 [2425.80697073 1332.18246961]
 [ 532.05923408 1851.13548155]
 [1360.97731988 2003.33318839]
 [2300.3140697  2129.16342521]
 [1190.8930267  2833.22649886]
 [2125.51651309 3012.87283858]]

Template centers, in um:
[[   0.    0.   32.]
 [1000.    0.   34.]
 [2000.    0.   32.]
 [   0. 1000.   32.]
 [1000. 1000.   34.]
 [2000. 1000.   34.]
 [1000. 2000.   34.]
 [2000. 2000.   36.]]

Transformation coefficients:
Vector u: [ 1.09103890e+00 -1.89301644e-01  5.38955870e-04]
Vector v: [0.19078772 1.17072692 0.00127879]
Vector h: [ -883.29920113 -1092.58880195    30.22899649]


In [ ]:
in_plane_err = multi_plane.avg_in_plane_projection_error()
print("Average in-plane projection error:", in_plane_err)

out_plane_err = multi_plane.avg_out_of_plane_projection_error()
print("Average out-of-plane projection error:", out_plane_err)

Average in-plane projection error: 57.264813072846685
(8,)
Average in-plane projection error: 33.500000000000014
